In [92]:
import torch
import torch.nn as nn
import pandas as pd

In [93]:
# --- Load input features from CSV ---
df = pd.read_csv("../hansa_test.csv")  
input_tensor = torch.tensor(df.values, dtype=torch.float32)  # shape: (N, features)

In [94]:
# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, outputs):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, outputs)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)  
        # or:
        #out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        out = self.fc(out)
        # out: (n, 10)
        return out

In [95]:
# Hyper-parameters 
# input_size = 784 # 28x28
device = torch.device("cpu")
# Hyper-parameters 
# input_size = 784 # 28x28
output_size = 1
num_epochs = 2
batch_size = 1000
learning_rate = 0.001

input_size = 3
sequence_length = 10002
hidden_size = 100
num_layers = 1

In [96]:
# ----------- Load the model ------------------------------------
model = RNNRegressor(input_size, hidden_size, num_layers, output_size).to(device)
model.load_state_dict(torch.load("trained_rnn.pth"))
model.eval()

RNNRegressor(
  (rnn): RNN(3, 100, batch_first=True)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

In [97]:

# ----------- Read input CSV ------------------------------------
df = pd.read_csv("../hansa_test.csv")  
X_new = df[['Alpha', 'q', 'delta_e']].values
X_new_tensor = torch.tensor(X_new, dtype=torch.float32).unsqueeze(1).to(device)  # (batch, se


In [98]:
# ----------- Make Predictions ----------------------------------
with torch.no_grad():
    predictions = model(X_new_tensor)
    predictions = predictions.cpu().numpy().flatten()  # Convert to 1D array

In [99]:
# ----------- Save to output CSV -------------------------------
df['Predicted_Cd'] = predictions
df.to_csv("predicted_rnn.csv", index=False)

print("Predictions saved to predicted_rnn.csv")

Predictions saved to predicted_rnn.csv
